In [1]:
import os 
import pandas as pd
import numpy as np
import math


In [2]:


def chk_unique_eid(df):
    """
    Check unique eid number for a dataframe
    """
    print('loaded df has unique eid count: '+ str(len(df.eid.unique())))

    
    

def lst_ind(dfa_list,ind_val):
    """
    return a list of icd code that match with 'ind_val'
    """
    
    pre0=[]
    for i in dfa_list:
        if pd.isnull(i):
            pre0.append([])
        elif pd.notnull(i):
            si=[]
            jl=i.split(',')
            for ei in jl:
                ef=ei.replace(',','')
                efa,efb,efc=ef.partition(str(ind_val))
                if efa=='':
                    si.append(ef)
            pre0.append(si)
    return pre0

def hes_gen_ind_raw(icd,hesin_dfin,key_code,evnt, detail=False):
    """
    return a dataframe that contains indicator variable for a specific 'key_code'
    """

    #dfc=load_data_by_fid(fid_int)
    #df_icd9m=dfc.copy()
    #dfa=hesin[['eid',str(icd)]].copy()
    dfa=hesin_dfin[['eid','record_id',str(icd)]].copy()


    
    dfa_lst=dfa[dfa.columns[dfa.columns.get_loc(str(icd))]].values.tolist()
    pre0=lst_ind(dfa_lst,str(key_code))

    gen_hes_name='hes_'+str(icd)+'_'+str(evnt)+str(key_code)
    gen_ind_name='ind_'+str(icd)+'_'+str(evnt)+str(key_code)

    dfa[str(gen_hes_name)]=pre0
    dfa[dfa.columns[dfa.columns.get_loc(str(gen_hes_name))]]=dfa[dfa.columns[dfa.columns.get_loc(str(gen_hes_name))]].apply(lambda y: np.nan if len(y)==0 else y )
    
    dfa[str(gen_ind_name)]=pre0
    dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]]=dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]].apply(lambda y: 0 if len(y)==0 else 1 )
    
    print('\nHES '+str(icd)+' ',str(evnt)+'('+str(key_code)+')'+' count: '+str(dfa[dfa.columns[dfa.columns.get_loc(str(gen_hes_name))]].count())+',\nFreq_tab \n'+str(dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]].value_counts()))
    dfb=dfa[['eid','record_id',str(gen_ind_name)]]   
    
    if detail==True:
        return dfa
    else:
        return dfb




def hes_gen_ind_list(icd_in, hesin_dfin, key_code_list, evt, detai=False):
    """
    return a dataframe that contains indicator variables for each specific 'key_code' in 'key_code_list'
    """
    dfcl=[]
    if detai==False:
        for l in key_code_list:
            df_l=hes_gen_ind_raw(icd_in,hesin_dfin, l, str(evt), detail=False)
            dfcl.append(df_l)
        dfcl_merge=pd.concat(dfcl,axis=1)
        dfcl_merge=dfcl_merge.loc[:,~dfcl_merge.columns.duplicated()]  # drop duplicated 'eid' columns
        return dfcl_merge

    if detai==True:
        for l in key_code_list:
            df_l=hes_gen_ind_raw(icd_in,hesin_dfin, l, str(evt), detail=True)
            dfcl.append(df_l)
        dfcl_merge=pd.concat(dfcl,axis=1)
        dfcl_merge=dfcl_merge.loc[:,~dfcl_merge.columns.duplicated()]  # drop duplicated 'eid' columns
        return dfcl_merge


In [3]:
os.chdir("/projects/ps-janssen3/dsci-pa/yhuan162/temp_project/HES")

#load icd main and secondary diagnosis and icd count info
hesin_ukbb_date_icd_count_ready=pd.read_csv('hesin_ukbb_date_icd_count_ready.tsv', sep='\t',low_memory=False)


In [13]:
hesin_ukbb_date_icd_count_ready.sort_values(by='eid')[['eid','record_id','diag_icd10','diag_icd9','diag_icd9_sec','diag_icd10_sec','any_yr','att_yr','diff_yr_cat']].head(10)


,eid,record_id,diag_icd10,diag_icd9,diag_icd9_sec,diag_icd10_sec,any_yr,att_yr,diff_yr_cat
708073,1000011,3352364,M169,NaN,NaN,NaN,2013.0,2008,3.0
708074,1000011,3352365,M7986,NaN,NaN,Z966,2013.0,2008,3.0
708083,1000011,4431886,I839,NaN,NaN,Z922,2006.0,2008,1.0
708072,1000011,3352363,R31,NaN,NaN,"Z530,N390,I10,E780,Z922",2010.0,2008,3.0
708075,1000011,3352367,R104,NaN,NaN,NaN,1999.0,2008,1.0
708076,1000011,4431777,R31,NaN,NaN,"I10,E780,Z922",2010.0,2008,3.0
708077,1000011,4431778,M674,NaN,NaN,I10,2008.0,2008,2.0
708079,1000011,4431780,K409,NaN,NaN,D176,1996.0,2008,1.0
708080,1000011,4431781,R31,NaN,NaN,"Z530,N390,I10,E780,Z922",2010.0,2008,3.0
708078,1000011,4431779,K409,NaN,NaN,"I10,Z922",2013.0,2008,3.0


In [11]:
hesin_ukbb_date_icd_count_ready.count()

eid                                 2577597
record_id                           2577597
admidate                            2287724
anagest                               10263
anasdate                              13121
cause_icd10                          101660
cause_icd10_nb                          208
diag_icd10                          2512419
diag_icd10_nb                          5244
diag_icd9                             52121
diag_icd9_nb                              0
disdate                             2172998
epiend                              2518087
epistart                            2520356
matage                                14321
neocare                             1848633
numbaby                               31070
numpreg                               31063
opdate                              1914276
oper4                               2386150
oper4_nb                                  0
operstat                            1372220
diag_icd9_sec                   

In [4]:
hesin_ukbb_work = hesin_ukbb_date_icd_count_ready.copy()

In [5]:
%%time

### indicaotrs for icd main diagnosis
hes_icd10m_RA=hes_gen_ind_list(icd_in='diag_icd10',hesin_dfin=hesin_ukbb_work, evt='RA', key_code_list=['M05','M06'])
hes_icd9m_RA =hes_gen_ind_list(icd_in='diag_icd9' ,hesin_dfin=hesin_ukbb_work, evt='RA', key_code_list=['7140','7141','7142'])

hes_icd10m_lupus=hes_gen_ind_list(icd_in='diag_icd10',hesin_dfin=hesin_ukbb_work, evt='lupus', key_code_list=['M321'])
hes_icd9m_lupus =hes_gen_ind_list(icd_in='diag_icd9' ,hesin_dfin=hesin_ukbb_work, evt='lupus', key_code_list=['7100'])

hes_icd10m_PA=hes_gen_ind_list(icd_in='diag_icd10',hesin_dfin=hesin_ukbb_work, evt='PA', key_code_list=['L405'])
hes_icd9m_PA =hes_gen_ind_list(icd_in='diag_icd9' ,hesin_dfin=hesin_ukbb_work, evt='PA', key_code_list=['6960'])

### indicaotrs for icd secondary diagnosis
hes_icd10s_RA=hes_gen_ind_list(icd_in='diag_icd10_sec',hesin_dfin=hesin_ukbb_work, evt='RA', key_code_list=['M05','M06'])
hes_icd9s_RA =hes_gen_ind_list(icd_in='diag_icd9_sec' ,hesin_dfin=hesin_ukbb_work, evt='RA', key_code_list=['7140','7141','7142'])

hes_icd10s_lupus=hes_gen_ind_list(icd_in='diag_icd10_sec',hesin_dfin=hesin_ukbb_work, evt='lupus', key_code_list=['M321'])
hes_icd9s_lupus =hes_gen_ind_list(icd_in='diag_icd9_sec' ,hesin_dfin=hesin_ukbb_work, evt='lupus', key_code_list=['7100'])

hes_icd10s_PA=hes_gen_ind_list(icd_in='diag_icd10_sec',hesin_dfin=hesin_ukbb_work, evt='PA', key_code_list=['L405'])
hes_icd9s_PA =hes_gen_ind_list(icd_in='diag_icd9_sec' ,hesin_dfin=hesin_ukbb_work, evt='PA', key_code_list=['6960'])



HES diag_icd10  RA(M05) count: 2386,
Freq_tab 
0    2575211
1       2386
Name: ind_diag_icd10_RAM05, dtype: int64

HES diag_icd10  RA(M06) count: 13644,
Freq_tab 
0    2563953
1      13644
Name: ind_diag_icd10_RAM06, dtype: int64

HES diag_icd9  RA(7140) count: 122,
Freq_tab 
0    2577475
1        122
Name: ind_diag_icd9_RA7140, dtype: int64

HES diag_icd9  RA(7141) count: 1,
Freq_tab 
0    2577596
1          1
Name: ind_diag_icd9_RA7141, dtype: int64

HES diag_icd9  RA(7142) count: 0,
Freq_tab 
0    2577597
Name: ind_diag_icd9_RA7142, dtype: int64

HES diag_icd10  lupus(M321) count: 225,
Freq_tab 
0    2577372
1        225
Name: ind_diag_icd10_lupusM321, dtype: int64

HES diag_icd9  lupus(7100) count: 37,
Freq_tab 
0    2577560
1         37
Name: ind_diag_icd9_lupus7100, dtype: int64

HES diag_icd10  PA(L405) count: 1679,
Freq_tab 
0    2575918
1       1679
Name: ind_diag_icd10_PAL405, dtype: int64

HES diag_icd9  PA(6960) count: 3,
Freq_tab 
0    2577594
1          3
Name: ind_diag_

In [6]:
### Get RA/lupus/PA indicators

hes_icd_RA_ind_list=[hes_icd10m_RA, hes_icd10s_RA ,hes_icd9m_RA, hes_icd9s_RA]
hes_icd_RA_ind_pre=pd.concat(hes_icd_RA_ind_list, axis=1)
hes_icd_RA_ind_pre=hes_icd_RA_ind_pre.loc[:,~hes_icd_RA_ind_pre.columns.duplicated()]
hes_icd_RA_ind=hes_icd_RA_ind_pre[['eid','record_id']].copy()
hes_icd_RA_ind['hes_icd_RA_ind']=hes_icd_RA_ind_pre.drop(['eid','record_id'],axis=1).sum(axis=1)
hes_icd_RA_ind.hes_icd_RA_ind=hes_icd_RA_ind.hes_icd_RA_ind.apply(lambda y: 1 if y>0 else y)

hes_icd_lupus_ind_list=[hes_icd10m_lupus, hes_icd10s_lupus ,hes_icd9m_lupus, hes_icd9s_lupus]
hes_icd_lupus_ind_pre=pd.concat(hes_icd_lupus_ind_list, axis=1)
hes_icd_lupus_ind_pre=hes_icd_lupus_ind_pre.loc[:,~hes_icd_lupus_ind_pre.columns.duplicated()]
hes_icd_lupus_ind=hes_icd_lupus_ind_pre[['eid','record_id']].copy()
hes_icd_lupus_ind['hes_icd_lupus_ind']=hes_icd_lupus_ind_pre.drop(['eid','record_id'],axis=1).sum(axis=1)
hes_icd_lupus_ind.hes_icd_lupus_ind=hes_icd_lupus_ind.hes_icd_lupus_ind.apply(lambda y: 1 if y>0 else y)

hes_icd_PA_ind_list=[hes_icd10m_PA, hes_icd10s_PA ,hes_icd9m_PA, hes_icd9s_PA]
hes_icd_PA_ind_pre=pd.concat(hes_icd_PA_ind_list, axis=1)
hes_icd_PA_ind_pre=hes_icd_PA_ind_pre.loc[:,~hes_icd_PA_ind_pre.columns.duplicated()]
hes_icd_PA_ind=hes_icd_PA_ind_pre[['eid','record_id']].copy()
hes_icd_PA_ind['hes_icd_PA_ind']=hes_icd_PA_ind_pre.drop(['eid','record_id'],axis=1).sum(axis=1)
hes_icd_PA_ind.hes_icd_PA_ind=hes_icd_PA_ind.hes_icd_PA_ind.apply(lambda y: 1 if y>0 else y)



In [7]:
### Add RA/lupus/PA indicators to hesin_ukbb_work dataframe: save as 'RA_hesin_ukbb_work_temp'

hesin_ukbb_work_temp_list = [hesin_ukbb_work, hes_icd_RA_ind, hes_icd_lupus_ind, hes_icd_PA_ind]
hesin_ukbb_work_temp = pd.concat(hesin_ukbb_work_temp_list, axis=1)
hesin_ukbb_work_temp = hesin_ukbb_work_temp.loc[:,~hesin_ukbb_work_temp.columns.duplicated()]

In [8]:
hesin_ukbb_work_temp.to_csv('RA_hesin_ukbb_work_temp.csv', index=None)
hesin_ukbb_work_temp.to_pickle("/projects/ps-janssen3/dsci-pa/yhuan162/temp_project/HES/outcomes/RA/hesin_RA_wd.pkl")

### resume from work 

In [4]:

### load from pickle
hesin_ukbb_work_temp=pd.read_pickle("/projects/ps-janssen3/dsci-pa/yhuan162/temp_project/HES/outcomes/RA/hesin_RA_wd.pkl")

### or load from csv
#hesin_ukbb_work_temp=pd.read_csv('RA_hesin_ukbb_work_temp.csv',low_memory=False)



In [9]:
### load ukbb RA with icd dataframe
ukbb_with_RA_icd=pd.read_csv('/projects/ps-janssen3/dsci-pa/yhuan162/temp_project/ukbb/data/i0/varOutcome/RA/with_RA_icd.csv')
ukbb_with_RA_icd.columns = ['eid', 'ukbb_icd_RA_ind', 'ukbb_ct_icd_RA_ind', 'ukbb_icd_lupus_ind',
       'ukbb_ct_icd_lupus_ind', 'ukbb_icd_PA_ind', 'ukbb_ct_icd_PA_ind', 'ukbb_any_icd_ct']

In [10]:
### check RA_ind==1 in ukbb data and HES data

chk_unique_eid(ukbb_with_RA_icd)
chk_unique_eid(hesin_ukbb_work_temp[hesin_ukbb_work_temp.hes_icd_RA_ind==1])

chk2=hesin_ukbb_work_temp[hesin_ukbb_work_temp.hes_icd_RA_ind==1]

m1=ukbb_with_RA_icd[['eid']].drop_duplicates()
m2=chk2[['eid']].drop_duplicates()

m12=pd.merge(m1,m2,on='eid',how='outer')

print('\ntwo subject with RAind==1, which are exist in ukbb data but are not exist in HES data:')
nlst=m2.eid.tolist()
m1[~m1.eid.isin(nlst)]




loaded df has unique eid count: 5515
loaded df has unique eid count: 5513

two subject with RAind==1, which are exist in ukbb data but are not exist in HES data:


,eid
2833,3583105
3952,4582812


In [12]:
### merge hesin_ukbb_work_temp with ukbb_with_RA_icd
hesin_ukbb_work_temp1 = pd.merge(hesin_ukbb_work_temp, ukbb_with_RA_icd, on='eid', how='left')

In [14]:
### Get pre/at/post baseline dataframe from hesin_ukbb_work_temp1 

hesin_ukbb_work_pre_temp  = hesin_ukbb_work_temp1[hesin_ukbb_work_temp1.diff_yr_cat==1].copy()
hesin_ukbb_work_at_temp   = hesin_ukbb_work_temp1[hesin_ukbb_work_temp1.diff_yr_cat==2].copy()
hesin_ukbb_work_post_temp = hesin_ukbb_work_temp1[hesin_ukbb_work_temp1.diff_yr_cat==3].copy()

In [29]:

### Get RA/lupus/PA case count @ pre baseline
### RA count
hesin_ukbb_work_pre_temp_RA = hesin_ukbb_work_pre_temp[['eid', 'hes_icd_RA_ind']].sort_values(by=['eid']).copy()
hesin_ukbb_work_pre_temp1_RA = hesin_ukbb_work_pre_temp_RA.set_index(['eid'])
hesin_ukbb_work_pre_temp2_RA = hesin_ukbb_work_pre_temp1_RA.groupby(by=['eid']).sum().groupby(level=[0]).cumsum()
hesin_ukbb_work_pre_temp3_RA = hesin_ukbb_work_pre_temp2_RA.reset_index()
hesin_ukbb_work_pre_temp3_RA.columns = ['eid', 'hes_RA_reported_ct_pre']

### lupus count
hesin_ukbb_work_pre_temp_lupus = hesin_ukbb_work_pre_temp[['eid', 'hes_icd_lupus_ind']].sort_values(by=['eid']).copy()
hesin_ukbb_work_pre_temp1_lupus = hesin_ukbb_work_pre_temp_lupus.set_index(['eid'])
hesin_ukbb_work_pre_temp2_lupus = hesin_ukbb_work_pre_temp1_lupus.groupby(by=['eid']).sum().groupby(level=[0]).cumsum()
hesin_ukbb_work_pre_temp3_lupus = hesin_ukbb_work_pre_temp2_lupus.reset_index()
hesin_ukbb_work_pre_temp3_lupus.columns = ['eid', 'hes_lupus_reported_ct_pre']

### PA count
hesin_ukbb_work_pre_temp_PA = hesin_ukbb_work_pre_temp[['eid', 'hes_icd_PA_ind']].sort_values(by=['eid']).copy()
hesin_ukbb_work_pre_temp1_PA = hesin_ukbb_work_pre_temp_PA.set_index(['eid'])
hesin_ukbb_work_pre_temp2_PA = hesin_ukbb_work_pre_temp1_PA.groupby(by=['eid']).sum().groupby(level=[0]).cumsum()
hesin_ukbb_work_pre_temp3_PA = hesin_ukbb_work_pre_temp2_PA.reset_index()
hesin_ukbb_work_pre_temp3_PA.columns = ['eid', 'hes_PA_reported_ct_pre']

### merge RA/lupus/PA count data
hesin_ukbb_work_pre_ct_temp3_Rlu = pd.merge(hesin_ukbb_work_pre_temp3_RA, hesin_ukbb_work_pre_temp3_lupus, on='eid', how='outer')
hesin_ukbb_work_pre_ct_temp3_RluP = pd.merge(hesin_ukbb_work_pre_ct_temp3_Rlu, hesin_ukbb_work_pre_temp3_PA, on='eid', how='outer')

hesin_ukbb_work_pre_ct_temp4 = hesin_ukbb_work_pre_ct_temp3_RluP.fillna(0).copy()



In [33]:
hesin_ukbb_work_pre_ct_temp5 = hesin_ukbb_work_pre_temp[['eid', 'hes_icd_reported_ct_pre']].drop_duplicates().copy()

In [34]:

### Assume lab_rf==0 or 1 to get the lower/upper bound of possible numer of RA cases

hesin_ukbb_work_pre_ct_temp10 = pd.merge(hesin_ukbb_work_pre_ct_temp5, hesin_ukbb_work_pre_ct_temp4, on='eid', how='left')
hesin_ukbb_work_pre_ct_temp10['fake_lab_rf1']=np.repeat(1, len(hesin_ukbb_work_pre_ct_temp10.eid.unique()))
hesin_ukbb_work_pre_ct_temp10['fake_lab_rf0']=np.repeat(0, len(hesin_ukbb_work_pre_ct_temp10.eid.unique()))


In [38]:
hesin_ukbb_work_pre_ct_temp10['fk1_beta1']=hesin_ukbb_work_pre_ct_temp10['hes_RA_reported_ct_pre'].apply(lambda y: 1.937*math.log1p(y))
hesin_ukbb_work_pre_ct_temp10['fk1_beta2']=hesin_ukbb_work_pre_ct_temp10['hes_lupus_reported_ct_pre'].apply(lambda y: -0.529*math.log1p(y))
hesin_ukbb_work_pre_ct_temp10['fk1_beta3']=hesin_ukbb_work_pre_ct_temp10['hes_PA_reported_ct_pre'].apply(lambda y: -0.122*math.log1p(y))
hesin_ukbb_work_pre_ct_temp10['fk1_beta4']=hesin_ukbb_work_pre_ct_temp10['fake_lab_rf1'].apply(lambda y: 1.639*y)
hesin_ukbb_work_pre_ct_temp10['fk1_beta5']=hesin_ukbb_work_pre_ct_temp10['hes_icd_reported_ct_pre'].apply(lambda y: -0.954*math.log1p(y))

hesin_ukbb_work_pre_ct_temp10['fk0_beta1']=hesin_ukbb_work_pre_ct_temp10['hes_RA_reported_ct_pre'].apply(lambda y: 1.937*math.log1p(y))
hesin_ukbb_work_pre_ct_temp10['fk0_beta2']=hesin_ukbb_work_pre_ct_temp10['hes_lupus_reported_ct_pre'].apply(lambda y: -0.529*math.log1p(y))
hesin_ukbb_work_pre_ct_temp10['fk0_beta3']=hesin_ukbb_work_pre_ct_temp10['hes_PA_reported_ct_pre'].apply(lambda y: -0.122*math.log1p(y))
hesin_ukbb_work_pre_ct_temp10['fk0_beta4']=hesin_ukbb_work_pre_ct_temp10['fake_lab_rf0'].apply(lambda y: 1.639*y)
hesin_ukbb_work_pre_ct_temp10['fk0_beta5']=hesin_ukbb_work_pre_ct_temp10['hes_icd_reported_ct_pre'].apply(lambda y: -0.954*math.log1p(y))

hesin_ukbb_work_pre_ct_temp10['fk1_sumbeta']=hesin_ukbb_work_pre_ct_temp10['fk1_beta1']+hesin_ukbb_work_pre_ct_temp10['fk1_beta2']+hesin_ukbb_work_pre_ct_temp10['fk1_beta3']+hesin_ukbb_work_pre_ct_temp10['fk1_beta4']+hesin_ukbb_work_pre_ct_temp10['fk1_beta5']
hesin_ukbb_work_pre_ct_temp10['fk0_sumbeta']=hesin_ukbb_work_pre_ct_temp10['fk0_beta1']+hesin_ukbb_work_pre_ct_temp10['fk0_beta2']+hesin_ukbb_work_pre_ct_temp10['fk0_beta3']+hesin_ukbb_work_pre_ct_temp10['fk0_beta4']+hesin_ukbb_work_pre_ct_temp10['fk0_beta5']

intercept=-1.017

hesin_ukbb_work_pre_ct_temp10['fk1_score']=hesin_ukbb_work_pre_ct_temp10['fk1_sumbeta'].apply(lambda y: math.exp(intercept+y)/(1+math.exp(intercept+y)) )
hesin_ukbb_work_pre_ct_temp10['fk0_score']=hesin_ukbb_work_pre_ct_temp10['fk0_sumbeta'].apply(lambda y: math.exp(intercept+y)/(1+math.exp(intercept+y)) )

In [39]:
fk1_hesin_RA_case=hesin_ukbb_work_pre_ct_temp10[hesin_ukbb_work_pre_ct_temp10.fk1_score>=.632]
fk0_hesin_RA_case=hesin_ukbb_work_pre_ct_temp10[hesin_ukbb_work_pre_ct_temp10.fk0_score>=.632]

chk_unique_eid(fk1_hesin_RA_case)
chk_unique_eid(fk0_hesin_RA_case)

loaded df has unique eid count: 1898
loaded df has unique eid count: 417


In [40]:
### When lab data available, then process to get pre/at/post baseline RA case.